In [10]:
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]= os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"]= os.getenv("GOOGLE_API_KEY")

os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]= os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]= os.getenv("LANGCHAIN_TRACING_V2")
#os.environ["TAVILY_API_KEY"]= os.getenv("TAVILY_API_KEY")

model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=os.environ["GOOGLE_API_KEY"]
)

response = await model_client.create([UserMessage(content="What is Agentic AI in 1 sentence?", source="user")])

print(response)
await model_client.close()

finish_reason='stop' content='Agentic AI refers to AI systems capable of autonomous action and decision-making to achieve goals.\n' usage=RequestUsage(prompt_tokens=10, completion_tokens=20) cached=False logprobs=None thought=None


In [11]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key=os.environ["OPENAI_API_KEY"]
)
response = await model_client.create([UserMessage(content="What is Agentic AI in 1 sentence?", source="user")])

print(response)
await model_client.close()


finish_reason='stop' content='Agentic AI refers to artificial intelligence systems that possess the ability to act autonomously and make decisions based on their own understanding and objectives, rather than merely responding to human inputs.' usage=RequestUsage(prompt_tokens=18, completion_tokens=35) cached=False logprobs=None thought=None


# First Autogen Agent

In [12]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ["OPENAI_API_KEY"]


# We use `OpenAIChatCompletionClient` to link our agent to GPT-4. 
# In v0.4, this replaces the older `llm_config` approach.
model_client = OpenAIChatCompletionClient(model='gpt-4o-mini',api_key=api_key)

# The `AssistantAgent` is a conversational AI that can respond to tasks. 
# We’ll give it a name and connect it to our model.
assistant = AssistantAgent(name = 'my_assistant',model_client=model_client)

#We’ll use the `run` method—a simple way in v0.4 to get a response. Watch this!
result = await assistant.run(task = "Tell me a fun fact about OpenAI .")
print(result)

messages=[TextMessage(id='6d2c84bb-2c2b-4535-ae78-d7aedf0eade8', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 21, 38, 37, 822635, tzinfo=datetime.timezone.utc), content='Tell me a fun fact about OpenAI .', type='TextMessage'), TextMessage(id='965ef09f-e1b6-40b5-bce6-190684af786d', source='my_assistant', models_usage=RequestUsage(prompt_tokens=45, completion_tokens=83), metadata={}, created_at=datetime.datetime(2025, 8, 8, 21, 38, 40, 865446, tzinfo=datetime.timezone.utc), content='OpenAI was founded in December 2015 with the mission to ensure that artificial general intelligence (AGI) benefits all of humanity. What makes it fun is that OpenAI has transitioned from a non-profit to a "capped-profit" model, allowing investors to earn returns while still prioritizing their mission. This unconventional approach is aimed at navigating the balance between innovation and responsibility in AI development. \n\nTERMINATE', type='TextMessage')] stop_reaso

In [13]:
print(result.messages[-1].content)

OpenAI was founded in December 2015 with the mission to ensure that artificial general intelligence (AGI) benefits all of humanity. What makes it fun is that OpenAI has transitioned from a non-profit to a "capped-profit" model, allowing investors to earn returns while still prioritizing their mission. This unconventional approach is aimed at navigating the balance between innovation and responsibility in AI development. 

TERMINATE


In [14]:
result = await assistant.run(task = "Tell me what is 5*25")
print(result.messages[-1].content)

5 multiplied by 25 equals 125. 

TERMINATE


# Autogen Architecture

## 1. Introduction <a id='introduction'></a>
AutoGen v0.4 is a framework for building **multi-agent AI systems** with an **asynchronous, event-driven architecture**. Key improvements over previous versions include:
- A redesigned **Core** layer for scalability.
- **AgentChat**, a simplified high-level API for agent interactions.
- **Extensions** for integrating external tools and data.

Official Quickstart: [AgentChat Quickstart](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/quickstart.html)

## 2. Architecture Diagram <a id='diagram'></a>
![AutoGen v0.4 Architecture](https://devblogs.microsoft.com/autogen/wp-content/uploads/sites/86/2025/01/a-screenshot-of-a-computer-ai-generated-content-m-4.png)

The architecture is divided into three main sections:
- **APPS**: Pre-built and custom applications.
- **AG Framework**: Core components powering agents.
- **Developer Tools**: Tools for testing and building.

## 3. APPS Layer <a id='apps'></a>
### a. **Magentic-One** ( Pre built application by MS - Ignore)
- **Purpose**: A pre-built application demonstrating multi-agent collaboration (e.g., group problem-solving).
- **Role**: Serves as a reference implementation for developers.
- **GitHub Reference**: Explore the code at [microsoft/autogen](https://github.com/microsoft/autogen/tree/main/notebook).

### b. **Your App Here** ( Here you build your application)
- **Purpose**: A customizable space for user-defined applications.
- **Use Cases**:
  - Customer service chatbots.
  - AI research assistants.
  - Automated content generation workflows.
- **Flexibility**: Integrates with LLMs (GPT-4, local models) via the AG Framework.

## 4. AG Framework Layer <a id='ag-framework'></a>
### a. **Core** ( It is foundation layer - Event driven , asynchronous)
- **Design**: Asynchronous, event-driven architecture using an **actor model**.
- **Key Features**:
  - Agents act independently and react to events (e.g., messages, API calls).
  - Supports parallel task execution for scalability.
  - Documentation: [Core Architecture](https://microsoft.github.io/autogen/stable/user-guide/architecture.html).

### b. **AgentChat** ( This is the API layer - used for task oriented agent building)
- **Purpose**: Simplifies building conversational agents.
- **Key Features**:
  - Predefined agent roles (e.g., `AssistantAgent`, `UserProxyAgent`).
  - Handles message routing and state management.
  - Documentation: [AgentChat Guide](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/introduction.html).

### c. **Extensions** ( adding flexibility to connect with external data and tools )
- **Purpose**: Extend functionality via plugins.
- **Types**:
  - **Official Extensions**: OpenAI/Azure connectors, math tools.
  - **Custom Extensions**: Integrate APIs (e.g., weather data, databases).
- **Documentation**: [Extensions Guide](https://microsoft.github.io/autogen/stable/user-guide/extensions.html).

## 5. Developer Tools <a id='developer-tools'></a>
### a. **Bench**
- **Purpose**: Benchmark agent performance and scalability.
- **Use Cases**:
  - Measure latency under heavy loads.
  - Optimize resource usage (e.g., API costs).
- **Documentation**: [Performance Testing](https://microsoft.github.io/autogen/stable/user-guide/performance.html).

### b. **Studio**
- **Purpose**: No-code/low-code agent design.
- **Features**:
  - Drag-and-drop workflow builder.
  - Interactive sandbox for testing.
  - Visual debugging tools.
- **Documentation**: [Studio Guide](https://microsoft.github.io/autogen/stable/user-guide/studio.html).

## 6. Summary Table <a id='summary'></a>
| **Component**   | **Key Role**                                                                 | **Documentation**                                                                 |
|------------------|-----------------------------------------------------------------------------|-----------------------------------------------------------------------------------|
| **Magentic-One** | Demo app for multi-agent collaboration.                                     | [Examples](https://microsoft.github.io/autogen/stable/user-guide/examples.html)   |
| **Your App Here** | Custom applications (e.g., chatbots, assistants).                         | [Quickstart](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/quickstart.html) |
| **Core**         | Async, event-driven foundation for scalable agents.                        | [Architecture](https://microsoft.github.io/autogen/stable/user-guide/architecture.html) |
| **AgentChat**    | Simplifies building conversational agents.                                 | [AgentChat Guide](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/introduction.html) |
| **Extensions**   | Adds external tools/APIs to agents.                                        | [Extensions](https://microsoft.github.io/autogen/stable/user-guide/extensions.html) |
| **Bench**        | Tests agent performance under load.                                        | [Performance](https://microsoft.github.io/autogen/stable/user-guide/performance.html) |
| **Studio**       | Visual tool for designing agent workflows.                                 | [Studio](https://microsoft.github.io/autogen/stable/user-guide/studio.html) |

## AutoGen vs. LangChain/ Langgraph <a id='autogen-langchain'></a>
### Key Differences
- **Architecture**: AutoGen uses an **event-driven, async architecture**, while LangChain relies on a **chain-based, sequential execution model**.
- **Multi-Agent Capabilities**: AutoGen excels in **autonomous multi-agent collaboration**, whereas LangChain focuses on **single-agent workflows with structured steps**.
- **Observability**: AutoGen provides **superior event tracking** for agent reasoning, while LangChain offers **basic tracing**.
- **Scalability**: AutoGen’s async foundation supports **complex agent networks**, while LangChain’s synchronous design struggles with **large-scale systems**.
- **Use Case Suitability**: AutoGen is ideal for **collaborative AI** (e.g., research teams), while LangChain is better for **data processing** or **question-answering**.

For more details, visit the official homepage: [LangChain](https://python.langchain.com/docs/).

## AutoGen vs. CrewAI <a id='autogen-crewai'></a>
### Key Differences
- **Flexibility**: AutoGen offers both **low-level Core API** and **high-level AgentChat API**, while CrewAI provides a **prescriptive, template-based model**.
- **Customization**: AutoGen allows **deep customization** of agent behavior, whereas CrewAI is better for **quick setups** with limited flexibility.
- **Maturity**: AutoGen has a **longer history** and **larger community**, while CrewAI is newer with a **growing user base**.
- **Learning Curve**: CrewAI is **simpler** for basic multi-agent tasks, while AutoGen requires **more learning** for advanced scenarios.
- **Use Case Suitability**: CrewAI suits **straightforward team tasks** (e.g., content creation), while AutoGen handles **intricate, dynamic AI models** (e.g., enterprise workflows).

For more details, visit the official homepage: [CrewAI](https://docs.crewai.com/).